In [ ]:
import tensorflow as tf 
from Vnet_multiclass import CreateVnet, FillWithBone
import os
from sklearn.model_selection import train_test_split
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
rootDirPath = "..\\mmwhs\\ct_train"

In [ ]:
a = np.array(nib.load("E:\isef-vascular\VascuIAR\DeepLearning\mmwhs\ct_train\ct_train_1001_image.nii.gz").get_fdata())

with open(".\\temp.txt", "w") as f: 
    f.write("First view\n")
    for i in range(a.shape[0]): 
        u = np.unique(a[i, :, :])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

    f.write("\nSecond view\n")
    for i in range(a.shape[1]): 
        u = np.unique(a[:, i, :])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

    f.write("\nThird view\n")
    for i in range(a.shape[2]): 
        u = np.unique(a[:, :, i])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

### EACH VIEW HAS DIFFERENT UNIQUE VALUES => INCONSISTENT WHEN TRYING TO REMOVE SLICES WITHOUT HEART + BY REMOVING SLICES WITHOUT HEART, IT WEAKENS THE MODEL'S ABILITY TO DIFFRENTIATE STRUCTUERS LIKE BONES / AIR IN SLICES CONTAINING HEART. 

# => 2 SOLUTIONS: 
#   1) Fill labels with zeros and real data with bone structures to make everything the same size
#   2) Crop all slices without heart to make it the same size  

# Cropping and Adding all add biases and unbalance to the dataset, except cropping is much harder to do homologously => Fill and offset bias 

In [ ]:
X = []
Y = []
target = 364
# target is 384 to allow Vnet to compress and decompress

pathList = os.listdir(rootDirPath)
for i in range(0, len(pathList) - 1, 2): 
    imagePath = os.path.join(rootDirPath, pathList[i])
    labelPath = os.path.join(rootDirPath, pathList[i + 1])

    image = np.array(nib.load(imagePath).get_fdata())
    label = np.array(nib.load(labelPath).get_fdata())

    image, label = FillWithBone(image, label, target)

    image = np.expand_dims(image, -1)
    label = np.expand_dims(label, -1)

    print(f"Image shape: {image.shape} || Path: {imagePath}")
    print(f"Label shape : {label.shape} || Path: {labelPath}")

    X.append(image)
    Y.append(label)



del pathList
del target 
del image
del label 

X = np.array(X)
Y = np.array(Y)

print(len(X), len(Y))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

print(len(X_train), len(X_test), len(Y_train), len(Y_test))

In [ ]:
model = CreateVnet()

In [ ]:
model.fit(X_train, Y_train, epochs=10, verbose=2)